<a href="https://colab.research.google.com/github/rekcahpassyla/nlp-project-2021/blob/main/NLP_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load prerequisites

In [1]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses
!pip install transformers

     |████████████████████████████████| 133kB 4.3MB/s 
     |████████████████████████████████| 1.2MB 32.2MB/s 
     |████████████████████████████████| 901kB 29.3MB/s 
     |████████████████████████████████| 7.5MB 39.8MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
ERROR: botocore 1.20.72 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 2.3MB 5.4MB/s 
     |████████████████████████████████| 3.3MB 15.3MB/s 


The next cell loads our trained model finetunings from our OSF repository. Only vanilla models are loaded; the profanity-enriched models require the installation of another library, `profanity_checking` that is difficult to configure for Colab because it requires a very specific out of date version of scikit-learn. 


In [2]:
!pip install osfclient
!osf -p 29nma fetch /osfstorage/results_ucl/uk_bert-base-uncased.pt
!osf -p 29nma fetch /osfstorage/results_ucl/us_bert-base-uncased.pt
!osf -p 29nma fetch /osfstorage/results_ucl/all_bert-base-uncased.pt

100% 438M/438M [00:09<00:00, 44.2Mbytes/s]
100% 438M/438M [00:11<00:00, 38.5Mbytes/s]
100% 438M/438M [00:10<00:00, 41.7Mbytes/s]


This cell contains code to load pretrained `bert-base-uncased` models, then load our fine-tunings for UK-, US- and combined-trained models. 

In [3]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# Change this if desired
DEVICE = torch.device('cpu')

class SarcasmDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

bert_type = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(bert_type)

uk_trained = AutoModelForSequenceClassification.from_pretrained(bert_type)
us_trained = AutoModelForSequenceClassification.from_pretrained(bert_type)
combined_trained = AutoModelForSequenceClassification.from_pretrained(bert_type)

# load finetunings
uk_trained.load_state_dict(torch.load('uk_bert-base-uncased.pt', map_location=DEVICE))
us_trained.load_state_dict(torch.load('us_bert-base-uncased.pt', map_location=DEVICE))
combined_trained.load_state_dict(torch.load('all_bert-base-uncased.pt', map_location=DEVICE))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

# Evaluation

The next cell contains code to evaluate the model on data. The data must be in the form of a list of dictionaries, see the docstring of `evaluate` for more details. 


In [5]:
def evaluate(model, data):
    # data must be a list of dictionaries in this format
    # [
    #   {'headline': 'Man bites dog', 'is_sarcastic': 1},
    #   {'headline': 'Dog bites man', 'is_sarcastic': 0},
    # ]
    headlines = [item['headline'] for item in data]
    labels = [item['is_sarcastic'] for item in data]
    encodings = tokenizer(headlines, truncation=True, padding=True)
    dataset = SarcasmDataset(encodings, labels)
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    preds = []
    labels = []
    for step, batch in enumerate(loader):
      with torch.no_grad():
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels += batch["labels"].numpy().tolist()
            this_labels = batch["labels"].to(DEVICE)
            outputs = model(input_ids, attention_mask=attention_mask, labels=this_labels)
            this_preds = outputs.logits.argmax(axis=1)
            preds += this_preds.cpu().numpy().tolist()
            loss = outputs[0]
    preds = np.array(preds)
    labels = np.array(labels)
    accuracy = (preds == labels).mean()
    return preds, labels, accuracy

def evaluate_uk(data):
  return evaluate(uk_trained, data)

def evaluate_us(data):
  return evaluate(us_trained, data)

def evaluate_combined(data):
  return evaluate(combined_trained, data)

DATA = [
       {'headline': 'One Covid vaccine dose halves transmission - study', 'is_sarcastic': 0},
       {'headline': 'Six fun things to do with your Brexit 50p', 'is_sarcastic': 1},
]

print("------------------")
print("UK-trained: ")
preds, labels, accuracy = evaluate_uk(DATA)

for i, item in enumerate(DATA):
  print(f"Headline: {item['headline']}")
  print(f"Actual category: {item['is_sarcastic']}" % item)
  print(f"Predicted category: {preds[i]}")
  print()

print(f"Total accuracy: {accuracy}")

print("------------------")
print("US-trained: ")
preds, labels, accuracy = evaluate_us(DATA)

for i, item in enumerate(DATA):
  print(f"Headline: {item['headline']}")
  print(f"Actual category: {item['is_sarcastic']}" % item)
  print(f"Predicted category: {preds[i]}")
  print()

print(f"Total accuracy: {accuracy}")
print(f"")

print("------------------")
print("Combined trained: ")
preds, labels, accuracy = evaluate_combined(DATA)

for i, item in enumerate(DATA):
  print(f"Headline: {item['headline']}")
  print(f"Actual category: {item['is_sarcastic']}" % item)
  print(f"Predicted category: {preds[i]}")
  print()

print(f"Total accuracy: {accuracy}")

------------------
UK-trained: 
Headline: One Covid vaccine dose halves transmission - study
Actual category: 0
Predicted category: 1

Headline: Six fun things to do with your Brexit 50p
Actual category: 1
Predicted category: 1

Total accuracy: 0.5
------------------
US-trained: 
Headline: One Covid vaccine dose halves transmission - study
Actual category: 0
Predicted category: 1

Headline: Six fun things to do with your Brexit 50p
Actual category: 1
Predicted category: 0

Total accuracy: 0.0

------------------
Combined trained: 
Headline: One Covid vaccine dose halves transmission - study
Actual category: 0
Predicted category: 1

Headline: Six fun things to do with your Brexit 50p
Actual category: 1
Predicted category: 0

Total accuracy: 0.0
